In [4]:
import os
import shutil
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm # 작업 진행률 표시

In [5]:
# 1.1 클래스 및 경로 설정
# (주의: 이 경로는 사용자가 AI Hub 데이터를 다운로드하고 압축 해제한 경로로 수정해야 합니다)
original_data_path = "../datasets/" # ⬅️ **AI Hub 원본 데이터 경로**
original_images_path = os.path.join(original_data_path, "image")
original_labels_path = os.path.join(original_data_path, "json") # JSON 파일이 있는 곳

# 1.2 클래스 정의 (매우 중요!)
# 이 순서가 class_index가 됩니다 (0부터 8까지)
CLASS_NAMES = [
    'apple_special', 'apple_high', 'apple_medium',
    'pear_special', 'pear_high', 'pear_medium',
    'persimmon_special', 'persimmon_high', 'persimmon_medium'
]

# 클래스 이름을 인덱스로 매핑하는 딕셔너리
class_map = {name: i for i, name in enumerate(CLASS_NAMES)}
print(f"클래스 맵: {class_map}")

# 1.3 어노테이션 변환 함수 (AI Hub JSON 구조에 맞게 수정 필요)

def convert_to_yolo_format(json_path, image_width, image_height):
    """
    AI Hub JSON 어노테이션을 YOLO .txt 형식의 문자열 리스트로 변환합니다.
    AI Hub JSON 파일의 정확한 구조에 따라 이 함수는 크게 달라질 수 있습니다.
    """
    yolo_labels = []
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # (가정) JSON 안에 이미지 크기 정보가 있다고 가정
        # img_width = data['image']['width']
        # img_height = data['image']['height']
        
        # (가정) 'annotations' 리스트 안에 객체 정보가 있다고 가정
        for obj in data.get('annotations', []):
            
            # 1. 클래스 매핑 (가장 중요)
            # AI Hub 데이터의 '품목'과 '등급'을 조합
            fruit_type = obj.get('cate1') # 예: '사과'
            quality = obj.get('cate3') # 예: '특'
            
            # (예시) AI Hub 데이터를 기반으로 클래스 이름 생성 (정확한 키값 확인 필요)
            combined_class = ""
            if fruit_type == '사과':
                if quality == '특': combined_class = 'apple_special'
                elif quality == '상': combined_class = 'apple_high'
                elif quality == '중': combined_class = 'apple_medium'
            elif fruit_type == '배':
                # ... 배에 대한 로직 ...
                combined_class = 'pear_special' # 임시
            elif fruit_type == '감':
                # ... 감에 대한 로직 ...
                combined_class = 'persimmon_special' # 임시

            # 2. 클래스 인덱스 찾기
            if combined_class in class_map:
                class_index = class_map[combined_class]
            else:
                continue # 9개 클래스 외에는 무시

            # 3. Bbox 좌표 변환 (JSON의 bbox 형식에 따라 달라짐)
            # 예: AI Hub가 [xmin, ymin, xmax, ymax] 형식으로 제공한다면
            bbox = obj.get('bbox') # [xmin, ymin, xmax, ymax] 형태라고 가정
            xmin, ymin, xmax, ymax = bbox
            
            x_center = ((xmin + xmax) / 2) / image_width
            y_center = ((ymin + ymax) / 2) / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height
            
            # 4. YOLO 형식 문자열 생성
            yolo_labels.append(f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
    except Exception as e:
        print(f"Error processing {json_path}: {e}")
        
    return yolo_labels

# 1.4 (실행) 데이터 변환 루프
# (실제 실행 시에는 이미지 크기를 얻기 위해 cv2.imread 또는 PIL.Image.open 사용 필요)
IMG_W, IMG_H = 1000, 1000 # ⬅️ **데이터셋의 실제 이미지 크기 확인 필요**

# YOLO 형식으로 변환된 라벨을 저장할 디렉터리
output_labels_path = "labels/" # ⬅️ **YOLO 라벨 저장할 새 폴더 경로**
os.makedirs(output_labels_path, exist_ok=True)

json_files = glob.glob(os.path.join(original_labels_path, "*.json"))

print("YOLO 라벨 변환 시작...")
for json_file in tqdm(json_files):
    base_filename = os.path.splitext(os.path.basename(json_file))[0]
    txt_filename = base_filename + ".txt"
    txt_filepath = os.path.join(output_labels_path, txt_filename)
    
    # 변환 함수 호출 (실제 이미지 크기 전달)
    yolo_data = convert_to_yolo_format(json_file, IMG_W, IMG_H) 
    
    if yolo_data:
        with open(txt_filepath, 'w', encoding='utf-8') as f:
            f.write("\n".join(yolo_data))

print("변환 완료.")

클래스 맵: {'apple_special': 0, 'apple_high': 1, 'apple_medium': 2, 'pear_special': 3, 'pear_high': 4, 'pear_medium': 5, 'persimmon_special': 6, 'persimmon_high': 7, 'persimmon_medium': 8}
YOLO 라벨 변환 시작...


0it [00:00, ?it/s]

변환 완료.


In [6]:
# 2.1 경로 설정
dataset_root = "dataset/" # ⬅️ **최종 데이터셋을 구성할 경로**
train_img_dir = os.path.join(dataset_root, "images", "train")
val_img_dir = os.path.join(dataset_root, "images", "val")
train_lbl_dir = os.path.join(dataset_root, "labels", "train")
val_lbl_dir = os.path.join(dataset_root, "labels", "val")

# 폴더 생성
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# 2.2 파일 목록 가져오기
# 원본 이미지 경로 (AI Hub 이미지 폴더)
original_images_path = "data/image" # ⬅️ **원본 이미지 폴더 경로**
# 변환된 라벨 경로 (1단계에서 생성한 .txt 폴더)
yolo_labels_path = "labels/" # ⬅️ **1단계에서 생성한 .txt 폴더 경로**

all_images = [f for f in os.listdir(original_images_path) if f.endswith(('.jpg', '.png'))]
all_labels = [f for f in os.listdir(yolo_labels_path) if f.endswith('.txt')]

# 이미지와 라벨 파일명이 일치하는지 확인 (이름 기준)
image_basenames = set([os.path.splitext(f)[0] for f in all_images])
label_basenames = set([os.path.splitext(f)[0] for f in all_labels])

# 이미지와 라벨 모두 존재하는 파일만 선택
valid_basenames = list(image_basenames.intersection(label_basenames))
print(f"유효한 이미지/라벨 쌍: {len(valid_basenames)} 개")

# 2.3 Train / Validation 분할 (80:20)
train_names, val_names = train_test_split(valid_basenames, test_size=0.2, random_state=42)

print(f"Train: {len(train_names)}, Validation: {len(val_names)}")

# 2.4 파일 복사 함수
def copy_files(filenames, img_src_dir, lbl_src_dir, img_dest_dir, lbl_dest_dir):
    for name in tqdm(filenames):
        # (주의: 원본 이미지 확장자를 알아야 함, .jpg로 가정)
        img_ext = '.jpg' # ⬅️ **원본 이미지 확장자 확인 필요 (.png일 수 있음)**
        lbl_ext = '.txt'
        
        # 원본 파일 경로
        img_src_path = os.path.join(img_src_dir, name + img_ext)
        lbl_src_path = os.path.join(lbl_src_dir, name + lbl_ext)
        
        # 대상 파일 경로
        img_dest_path = os.path.join(img_dest_dir, name + img_ext)
        lbl_dest_path = os.path.join(lbl_dest_dir, name + lbl_ext)
        
        # 복사
        if os.path.exists(img_src_path):
            shutil.copy(img_src_path, img_dest_path)
        if os.path.exists(lbl_src_path):
            shutil.copy(lbl_src_path, lbl_dest_path)

# 2.5 (실행) 파일 복사
print("Train 데이터 복사 중...")
copy_files(train_names, original_images_path, yolo_labels_path, train_img_dir, train_lbl_dir)

print("Validation 데이터 복사 중...")
copy_files(val_names, original_images_path, yolo_labels_path, val_img_dir, val_lbl_dir)

print("데이터 분할 및 복사 완료.")

FileNotFoundError: [Errno 2] No such file or directory: 'data/image'

In [ ]:
# 3.1 YAML 파일 내용 작성
# (주의: yolov5 폴더 내에 있어야 함. 현재 %cd yolov5 상태)

data_yaml_content = f"""
train: {os.path.abspath(train_img_dir)}
val: {os.path.abspath(val_img_dir)}

# 클래스 수
nc: {len(CLASS_NAMES)}

# 클래스 이름
names: {CLASS_NAMES}
"""

# 3.2 YAML 파일 저장
yaml_path = os.path.join("/content/yolov5/data", "fruit_quality.yaml") # ⬅️ **설정 파일 저장 위치**

with open(yaml_path, 'w', encoding='utf-8') as f:
    f.write(data_yaml_content)

print(f"YAML 파일 저장 완료: {yaml_path}")
print("--- YAML 내용 ---")
print(data_yaml_content)

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov5/data/fruit_quality.yaml'

In [ ]:
# (현재 경로는 /content/yolov5 이어야 함)
# !ls data/fruit_quality.yaml # 파일이 잘 생성되었는지 확인

# 4.1 모델 학습 시작
# (GPU 사용 가능한 런타임에서 실행해야 합니다)
# (학습 시간은 데이터셋 크기와 epoch에 따라 몇 시간 소요될 수 있습니다)

!python train.py --img 640 --batch 16 --epochs 100 \
    --data data/fruit_quality.yaml \
    --weights yolov5s.pt \
    --name fruit_quality_run

In [ ]:
# 5.1 학습 결과 시각화 (TensorBoard)
# %load_ext tensorboard
# %tensorboard --logdir runs/train

# 5.2 결과 이미지 확인 (results.png)
from IPython.display import Image
Image(filename='runs/train/fruit_quality_run/results.png', width=800)

In [ ]:
# 5.3 검증 데이터셋을 사용한 성능 평가 (detect.py)
# 학습에 사용된 best.pt 가중치로 검증(val) 이미지들에 대해 예측을 실행합니다.
!python detect.py --weights runs/train/fruit_quality_run/weights/best.pt \
    --img 640 \
    --source {os.path.abspath(val_img_dir)} \
    --name val_inference

# 결과는 runs/detect/val_inference 폴더에 저장됩니다.
# 저장된 이미지들을 확인하여 모델이 'apple_high', 'pear_medium' 등을 잘 맞추는지 확인합니다.